In [100]:
# gets the html for soup to look at
from bs4 import BeautifulSoup
import requests

URL = 'https://www.biblegateway.com/passage/?search=Genesis+51&version=NRSV'
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

result = soup.find_all('td', 'class' == 'book-name')

book_num = {}

book = ''
max_chapter = 0

for i in range(len(result)):
    if i % 2 == 0:
        book = result[i].text
    else:
        num_list = result[i].text.split(' ')[-2]
        max_chapter = int(num_list)
        book_num[book] = max_chapter
        
book_num

{'Genesis': 50,
 'Exodus': 40,
 'Leviticus': 27,
 'Numbers': 36,
 'Deuteronomy': 34,
 'Joshua': 24,
 'Judges': 21,
 'Ruth': 4,
 '1 Samuel': 31,
 '2 Samuel': 24,
 '1 Kings': 22,
 '2 Kings': 25,
 '1 Chronicles': 29,
 '2 Chronicles': 36,
 'Ezra': 10,
 'Nehemiah': 13,
 'Esther': 10,
 'Job': 42,
 'Psalm': 150,
 'Proverbs': 31,
 'Ecclesiastes': 12,
 'Song of Solomon': 8,
 'Isaiah': 66,
 'Jeremiah': 52,
 'Lamentations': 5,
 'Ezekiel': 48,
 'Daniel': 12,
 'Hosea': 14,
 'Joel': 3,
 'Amos': 9,
 'Obadiah': 1,
 'Jonah': 4,
 'Micah': 7,
 'Nahum': 3,
 'Habakkuk': 3,
 'Zephaniah': 3,
 'Haggai': 2,
 'Zechariah': 14,
 'Malachi': 4,
 'Matthew': 28,
 'Mark': 16,
 'Luke': 24,
 'John': 21,
 'Acts': 28,
 'Romans': 16,
 '1 Corinthians': 16,
 '2 Corinthians': 13,
 'Galatians': 6,
 'Ephesians': 6,
 'Philippians': 4,
 'Colossians': 4,
 '1 Thessalonians': 5,
 '2 Thessalonians': 3,
 '1 Timothy': 6,
 '2 Timothy': 4,
 'Titus': 3,
 'Philemon': 1,
 'Hebrews': 13,
 'James': 5,
 '1 Peter': 5,
 '2 Peter': 3,
 '1 John': 

In [138]:
import pandas as pd
bible_book_list = []
for book in book_num:
    max_chap = book_num[book]

    df_list = []
    for chapter in range(1, max_chap + 1):

        URL = 'https://www.biblegateway.com/passage/?search=' + book + '+' + str(chapter) + '&version=NRSV'

        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html.parser")

        verse = []
        result = soup.find_all('span', 'class' == True)
        for r in result:
            if r.has_key('class') and 'text' in r['class']:
                verse.append(r.text.replace(u'\xa0', u' ').strip())
        df = pd.DataFrame(verse, columns = ['raw_verse'])


        df['Chapter'] = chapter
        df['Book'] = book
        df['verse_num'] = [t.split(' ', 1)[0] for t in df['raw_verse']]
        df = df[df['verse_num'].apply(lambda x: str(x).isdigit())]

        df['cleaned_verse'] = [t.split(' ', 1)[1] for t in df['raw_verse']]
        df_list.append(df)
    bible_book = pd.concat(df_list).reset_index()
    bible_book_list.append(bible_book)
    print('Finished scraping ', book)

/opt/homebrew/lib/python3.9/site-packages/bs4/element.py:1970: UserWarning: has_key is deprecated. Use has_attr("class") instead.
  warnings.warn('has_key is deprecated. Use has_attr("%s") instead.' % (


Finished scraping  Genesis
Finished scraping  Exodus
Finished scraping  Leviticus
Finished scraping  Numbers
Finished scraping  Deuteronomy
Finished scraping  Joshua
Finished scraping  Judges
Finished scraping  Ruth
Finished scraping  1 Samuel
Finished scraping  2 Samuel
Finished scraping  1 Kings
Finished scraping  2 Kings
Finished scraping  1 Chronicles
Finished scraping  2 Chronicles
Finished scraping  Ezra
Finished scraping  Nehemiah
Finished scraping  Esther
Finished scraping  Job
Finished scraping  Psalm
Finished scraping  Proverbs
Finished scraping  Ecclesiastes
Finished scraping  Song of Solomon
Finished scraping  Isaiah
Finished scraping  Jeremiah
Finished scraping  Lamentations
Finished scraping  Ezekiel
Finished scraping  Daniel
Finished scraping  Hosea
Finished scraping  Joel
Finished scraping  Amos
Finished scraping  Obadiah
Finished scraping  Jonah
Finished scraping  Micah
Finished scraping  Nahum
Finished scraping  Habakkuk
Finished scraping  Zephaniah
Finished scraping 

In [139]:
bible_book_list

[      index                                          raw_verse  Chapter  \
 0         6  1 In the beginning when God created[a] the hea...        1   
 1         7  2 the earth was a formless void and darkness c...        1   
 2         8  3 Then God said, “Let there be light”; and the...        1   
 3         9  4 And God saw that the light was good; and God...        1   
 4        10  5 God called the light Day, and the darkness h...        1   
 ...     ...                                                ...      ...   
 1528     28  22 So Joseph remained in Egypt, he and his fat...       50   
 1529     29  23 Joseph saw Ephraim’s children of the third ...       50   
 1530     30  24 Then Joseph said to his brothers, “I am abo...       50   
 1531     31  25 So Joseph made the Israelites swear, saying...       50   
 1532     32  26 And Joseph died, being one hundred ten year...       50   
 
          Book verse_num                                      cleaned_verse  
 0     G

In [140]:
bible_db = pd.concat(bible_book_list).reset_index()
bible_db = bible_db.drop(['level_0','index', 'raw_verse'], axis=1)
bible_db

,Chapter,Book,verse_num,cleaned_verse
0,1,Genesis,1,In the beginning when God created[a] the heave...
1,1,Genesis,2,the earth was a formless void and darkness cov...
2,1,Genesis,3,"Then God said, “Let there be light”; and there..."
3,1,Genesis,4,And God saw that the light was good; and God s...
4,1,Genesis,5,"God called the light Day, and the darkness he ..."
...,...,...,...,...
37656,18,4 Maccabees,20,O bitter was that day—and yet not bitter—when ...
37657,18,4 Maccabees,21,pierced the pupils of their eyes and cut out t...
37658,18,4 Maccabees,22,For these crimes divine justice pursued and wi...
37659,18,4 Maccabees,23,But the sons of Abraham with their victorious ...


In [142]:
bible_db.to_csv('bible_db.csv')  

In [83]:
# helper function to run sentiment analysis

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    sentiment_dict = sid_obj.polarity_scores(sentence)
     

    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        return("Positive")
 
    elif sentiment_dict['compound'] <= - 0.05 :
        return("Negative")
    else :
        return("Neutral")

In [144]:
bible_db['Sentiment'] = [sentiment_scores(verse) for verse in bible_db['cleaned_verse']]


,Chapter,Book,verse_num,cleaned_verse
0,1,Genesis,1,In the beginning when God created[a] the heave...
1,1,Genesis,2,the earth was a formless void and darkness cov...
2,1,Genesis,3,"Then God said, “Let there be light”; and there..."
3,1,Genesis,4,And God saw that the light was good; and God s...
4,1,Genesis,5,"God called the light Day, and the darkness he ..."
...,...,...,...,...
1545,1,Genesis,27,"So God created humankind[e] in his image,"
1546,1,Genesis,28,"God blessed them, and God said to them, “Be fr..."
1547,1,Genesis,29,"God said, “See, I have given you every plant y..."
1548,1,Genesis,30,"And to every beast of the earth, and to every ..."


In [145]:
bible_db[bible_db['Book'] == 'Luke']

,Chapter,Book,verse_num,cleaned_verse,Sentiment
24886,1,Luke,1,Since many have undertaken to set down an orde...,Positive
24887,1,Luke,2,just as they were handed on to us by those who...,Neutral
24888,1,Luke,3,"I too decided, after investigating everything ...",Positive
24889,1,Luke,4,so that you may know the truth concerning the ...,Positive
24890,1,Luke,5,"In the days of King Herod of Judea, there was ...",Neutral
...,...,...,...,...,...
26028,24,Luke,49,"And see, I am sending upon you what my Father ...",Positive
26029,24,Luke,50,"Then he led them out as far as Bethany, and, l...",Positive
26030,24,Luke,51,"While he was blessing them, he withdrew from t...",Positive
26031,24,Luke,52,"And they worshiped him, and[q] returned to Jer...",Positive


In [153]:
print([p for p in bible_db[(bible_db['Book'] == 'Luke') & (bible_db['Sentiment'] == 'Positive')]['cleaned_verse']])

['Since many have undertaken to set down an orderly account of the events that have been fulfilled among us,', 'I too decided, after investigating everything carefully from the very first,[a] to write an orderly account for you, most excellent Theophilus,', 'so that you may know the truth concerning the things about which you have been instructed.', 'Both of them were righteous before God, living blamelessly according to all the commandments and regulations of the Lord.', 'Once when he was serving as priest before God and his section was on duty,', 'Now at the time of the incense offering, the whole assembly of the people was praying outside.', 'You will have joy and gladness, and many will rejoice at his birth,', 'for he will be great in the sight of the Lord. He must never drink wine or strong drink; even before his birth he will be filled with the Holy Spirit.', 'He will turn many of the people of Israel to the Lord their God.', 'With the spirit and power of Elijah he will go before

In [162]:
luke = ([p for p in bible_db[(bible_db['Book'] == 'Luke') & (bible_db['Sentiment'] == 'Positive')]['cleaned_verse']])

In [1]:
import pandas as pd
bible_db = pd.read_csv ('bible_db.csv')

In [4]:
ecc = bible_db[bible_db['Book'] == 'Ecclesiastes'][:10]
ecc

,Unnamed: 0,Chapter,Book,verse_num,cleaned_verse
17316,17316,1,Ecclesiastes,1,"The words of the Teacher,[a] the son of David,..."
17317,17317,1,Ecclesiastes,2,"Vanity of vanities, says the Teacher,[b]"
17318,17318,1,Ecclesiastes,3,What do people gain from all the toil
17319,17319,1,Ecclesiastes,4,"A generation goes, and a generation comes,"
17320,17320,1,Ecclesiastes,5,"The sun rises and the sun goes down,"
17321,17321,1,Ecclesiastes,6,"The wind blows to the south,"
17322,17322,1,Ecclesiastes,7,"All streams run to the sea,"
17323,17323,1,Ecclesiastes,8,All things[c] are wearisome;
17324,17324,1,Ecclesiastes,9,"What has been is what will be,"
17325,17325,1,Ecclesiastes,10,"Is there a thing of which it is said,"


In [174]:
ecc

,Chapter,Book,verse_num,cleaned_verse,Sentiment
17316,1,Ecclesiastes,1,"The words of the Teacher,[a] the son of David,...",Neutral
17317,1,Ecclesiastes,2,"Vanity of vanities, says the Teacher,[b]",Negative
17318,1,Ecclesiastes,3,What do people gain from all the toil,Positive
17319,1,Ecclesiastes,4,"A generation goes, and a generation comes,",Neutral
17320,1,Ecclesiastes,5,"The sun rises and the sun goes down,",Neutral
17321,1,Ecclesiastes,6,"The wind blows to the south,",Neutral
17322,1,Ecclesiastes,7,"All streams run to the sea,",Neutral
17323,1,Ecclesiastes,8,All things[c] are wearisome;,Neutral
17324,1,Ecclesiastes,9,"What has been is what will be,",Neutral
17325,1,Ecclesiastes,10,"Is there a thing of which it is said,",Neutral


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))

In [5]:
for verse in ecc['cleaned_verse']:

    tokenized = sent_tokenize(verse)
    for i in tokenized:

        # Word tokenizers is used to find the words 
        # and punctuation in a string
        wordsList = nltk.word_tokenize(i)

        # removing stop words from wordList
        #wordsList = [w for w in wordsList if not w in stop_words] 

        #  Using a Tagger. Which is part-of-speech 
        # tagger or POS-tagger. 
        tagged = nltk.pos_tag(wordsList)

        print(tagged)

[('The', 'DT'), ('words', 'NNS'), ('of', 'IN'), ('the', 'DT'), ('Teacher', 'NNP'), (',', ','), ('[', 'VBZ'), ('a', 'DT'), (']', 'NN'), ('the', 'DT'), ('son', 'NN'), ('of', 'IN'), ('David', 'NNP'), (',', ','), ('king', 'VBG'), ('in', 'IN'), ('Jerusalem', 'NNP'), ('.', '.')]
[('Vanity', 'NN'), ('of', 'IN'), ('vanities', 'NNS'), (',', ','), ('says', 'VBZ'), ('the', 'DT'), ('Teacher', 'NNP'), (',', ','), ('[', 'NNP'), ('b', 'NN'), (']', 'NN')]
[('What', 'WP'), ('do', 'VBP'), ('people', 'NNS'), ('gain', 'VB'), ('from', 'IN'), ('all', 'PDT'), ('the', 'DT'), ('toil', 'NN')]
[('A', 'DT'), ('generation', 'NN'), ('goes', 'VBZ'), (',', ','), ('and', 'CC'), ('a', 'DT'), ('generation', 'NN'), ('comes', 'VBZ'), (',', ',')]
[('The', 'DT'), ('sun', 'NN'), ('rises', 'NNS'), ('and', 'CC'), ('the', 'DT'), ('sun', 'NN'), ('goes', 'VBZ'), ('down', 'RB'), (',', ',')]
[('The', 'DT'), ('wind', 'NN'), ('blows', 'VBZ'), ('to', 'TO'), ('the', 'DT'), ('south', 'NN'), (',', ',')]
[('All', 'DT'), ('streams', 'NNS')

In [9]:
exo = bible_db[bible_db['Book'] == 'Exodus']
print ([v for v in exo[exo.Chapter == 30]['cleaned_verse'] if 'census' in v])

['When you take a census of the Israelites to register them, at registration all of them shall give a ransom for their lives to the Lord, so that no plague may come upon them for being registered.']


In [10]:
print ([v for v in bible_db['cleaned_verse'] if 'census' in v])

['When you take a census of the Israelites to register them, at registration all of them shall give a ransom for their lives to the Lord, so that no plague may come upon them for being registered.', 'a beka a head (that is, half a shekel, measured by the sanctuary shekel), for everyone who was counted in the census, from twenty years old and upward, for six hundred three thousand, five hundred fifty men.', 'Take a census of the whole congregation of Israelites, in their clans, by ancestral houses, according to the number of names, every male individually;', 'Only the tribe of Levi you shall not enroll, and you shall not take a census of them with the other Israelites.', 'Take a census of the Kohathites separate from the other Levites, by their clans and their ancestral houses,', 'Take a census of the Gershonites also, by their ancestral houses and by their clans;', 'your dead bodies shall fall in this very wilderness; and of all your number, included in the census, from twenty years ol